# Week 5 - Vector Space Model (VSM) and Topic Modeling

Over the next weeks, we are going to re-implement Sherin's algorithm and apply it to the text data we've been working on last week! Here's our roadmap:

**Week 5 - data cleaning**
1. import the data
2. clean the data (e.g., remopve stop words, punctuation, etc.)
3. build a vocabulary for the dataset
4. create chunks of 100 words, with a 25-words overlap
5. create a word count matrix, where each chunk of a row and each column represents a word

**Week 6 - vectorization and linear algebra**
6. Dampen: weight the frequency of words (1 + log[count])
7. Scale: Normalize weighted frequency of words
8. Direction: compute deviation vectors

**Week 7 - Clustering**
9. apply different unsupervised machine learning algorithms
    * figure out how many clusters we want to keep
    * inspect the results of the clustering algorithm

**Week 8 - Visualizing the results**
10. create visualizations to compare documents

In [316]:
# in python code, our goal is to recreate the steps above as functions
# so that we can just one line to run topic modeling on a list of 
# documents: 
def ExtractTopicsVSM(documents, numTopics):
    ''' this functions takes in a list of documents (strings), 
        runs topic modeling (as implemented by Sherin, 2013)
        and returns the clustering results, the matrix used 
        for clustering a visualization '''
    
    # step 2: clean up the documents
    documents = clean_list_of_documents(documents)
    
    # step 3: let's build the vocabulary of these docs
    vocabulary = get_vocabulary(documents)
    
    # step 4: we build our list of 100-words overlapping fragments
    documents = flatten_and_overlap(documents)
    
    # step 5: we convert the chunks into a matrix
    matrix = docs_by_words_matrix(documents, vocabulary)
    
    # step 6: we weight the frequency of words (count = 1 + log(count))
    matrix = one_plus_log_mat(matrix, documents, vocabulary)
    
    # step 7: we normalize the matrix
    matrix = normalize(matrix)
    
    # step 8: we compute deviation vectors
    matrix = transform_deviation_vectors(matrix, documents)
    
    # step 9: we apply a clustering algorithm to find topics
    results_clustering = cluster_matrix(matrix)
    
    # step 10: we create a visualization of the topics
    visualization = visualize_clusters(results_clustering, vocabulary)
    
    # finally, we return the clustering results, the matrix, and a visualization
    return results_clustering, matrix, visualization

## Step 1 - Data Retrieval

In [317]:
# 1) using glob, find all the text files in the "Papers" folder
# Hint: refer to last week's notebook
import glob
papers = glob.glob("./Papers/paper*.txt")
print(papers)

['./Papers/paper12.txt', './Papers/paper5.txt', './Papers/paper4.txt', './Papers/paper13.txt', './Papers/paper11.txt', './Papers/paper6.txt', './Papers/paper7.txt', './Papers/paper10.txt', './Papers/paper14.txt', './Papers/paper3.txt', './Papers/paper2.txt', './Papers/paper15.txt', './Papers/paper0.txt', './Papers/paper1.txt', './Papers/paper16.txt', './Papers/paper9.txt', './Papers/paper8.txt']


In [318]:
# 2) get all the data from the text files into the "documents" list
# P.S. make sure you use the 'utf-8' encoding
documents = []
for paper in papers:
    file=open(paper,'r')
    documents.append(file.read())
documents[12]

"\x0czone out no more: mitigating mind wandering during\ncomputerized reading\nsidney k. d’mello, caitlin mills, robert bixler, & nigel bosch\nuniversity of notre dame\n118 haggar hall\nnotre dame, in 46556, usa\nsdmello@nd.edu\n\nabstract\nmind wandering, defined as shifts in attention from task-related\nprocessing to task-unrelated thoughts, is a ubiquitous\nphenomenon that has a negative influence on performance and\nproductivity in many contexts, including learning. we propose\nthat next-generation learning technologies should have some\nmechanism to detect and respond to mind wandering in real-time.\ntowards this end, we developed a technology that automatically\ndetects mind wandering from eye-gaze during learning from\ninstructional texts. when mind wandering is detected, the\ntechnology intervenes by posing just-in-time questions and\nencouraging re-reading as needed. after multiple rounds of\niterative refinement, we summatively compared the technology to\na yoked-control in a

In [319]:
# 3) print the first 1000 characters of the first document to see what it 
# looks like (we'll use this as a sanity check below)
print(documents[12][:1000])

zone out no more: mitigating mind wandering during
computerized reading
sidney k. d’mello, caitlin mills, robert bixler, & nigel bosch
university of notre dame
118 haggar hall
notre dame, in 46556, usa
sdmello@nd.edu

abstract
mind wandering, defined as shifts in attention from task-related
processing to task-unrelated thoughts, is a ubiquitous
phenomenon that has a negative influence on performance and
productivity in many contexts, including learning. we propose
that next-generation learning technologies should have some
mechanism to detect and respond to mind wandering in real-time.
towards this end, we developed a technology that automatically
detects mind wandering from eye-gaze during learning from
instructional texts. when mind wandering is detected, the
technology intervenes by posing just-in-time questions and
encouraging re-reading as needed. after multiple rounds of
iterative refinement, we summatively compared the technology to
a yoked-control in an experiment with 104 par

## Step 2 - Data Cleaning

In [320]:
# 4) only select the text that's between the first occurence of the 
# the word "abstract" and the last occurence of the word "reference"
# Optional: print the length of the string before and after, as a 
# sanity check
# HINT: https://stackoverflow.com/questions/14496006/finding-last-occurrence-of-substring-in-string-replacing-that
# read more about rfind: https://www.tutorialspoint.com/python/string_rfind.htm
abstract=[]
for doc in documents: 
    print(len(doc),end)
    start = doc.rfind('\nabstract\n')+len('abstract')
    end = doc.rfind('reference')
    doc = abstract.append(doc[start:end])

40387 40313
37214 35568
44037 33209
45258 40413
32277 42524
47851 28901
42617 41827
49177 35430
40655 43028
47377 33398
46761 43296
31574 42473
50043 28481
41110 39537
42046 35694
47845 37927
45724 44405


In [302]:
documents

["103\n\n\x0cepistemic network analysis and topic modeling for chat\ndata from collaborative learning environment\nzhiqiang cai\n\nbrendan eagan\n\nnia m. dowell\n\nthe university of memphis\n365 innovation drive, suite 410\nmemphis, tn, usa\n\nuniversity of wisconsin-madison\n1025 west johnson street\nmadison, wi, usa\n\nthe university of memphis\n365 innovation drive, suite 410\nmemphis, tn, usa\n\nzcai@memphis.edu\n\neaganb@gmail.com\n\nniadowell@gmail.com\n\njames w. pennebaker\n\ndavid w. shaffer\n\narthur c. graesser\n\nuniversity of texas-austin\n116 inner campus dr stop g6000\naustin, tx, usa\n\nuniversity of wisconsin-madison\n1025 west johnson street\nmadison, wi, usa\n\nthe university of memphis\n365 innovation drive, suite 403\nmemphis, tn, usa\n\npennebaker@utexas.edu\n\ndws@education.wisc.edu\n\nart.graesser@gmail.com\n\nabstract\nthis study investigates a possible way to analyze chat data from\ncollaborative learning environments using epistemic network\nanalysis and top

In [321]:
# 5) replace carriage returns (i.e., "\n") with a white space
# check that the result looks okay by printing the 
# first 1000 characters of the 1st doc:
for i,doc in enumerate(documents):
    documents[i] = doc.replace('\n', ' ')
    print(documents[12][:1000])

zone out no more: mitigating mind wandering during
computerized reading
sidney k. d’mello, caitlin mills, robert bixler, & nigel bosch
university of notre dame
118 haggar hall
notre dame, in 46556, usa
sdmello@nd.edu

abstract
mind wandering, defined as shifts in attention from task-related
processing to task-unrelated thoughts, is a ubiquitous
phenomenon that has a negative influence on performance and
productivity in many contexts, including learning. we propose
that next-generation learning technologies should have some
mechanism to detect and respond to mind wandering in real-time.
towards this end, we developed a technology that automatically
detects mind wandering from eye-gaze during learning from
instructional texts. when mind wandering is detected, the
technology intervenes by posing just-in-time questions and
encouraging re-reading as needed. after multiple rounds of
iterative refinement, we summatively compared the technology to
a yoked-control in an experiment with 104 par

In [322]:
# 6) replace the punctation below by a white space
# check that the result looks okay 
# (e.g., by print the first 1000 characters of the 1st doc)
import re
punctuation = ['.', '...', '!', '#', '"', '%', '$', "'", '&', ')', 
               '(', '+', '*', '-', ',', '/', '.', ';', ':', '=', 
               '<', '?', '>', '@', '",', '".', '[', ']', '\\', ',',
               '_', '^', '`', '{', '}', '|', '~', '−', '”', '“', '’']

for i in range(len(documents)):
    for p in punctuation:
        documents[i] = documents[i].replace(p, " ")
print(documents[12][:1000])

zone out no more  mitigating mind wandering during computerized reading sidney k  d mello  caitlin mills  robert bixler    nigel bosch university of notre dame 118 haggar hall notre dame  in 46556  usa sdmello nd edu  abstract mind wandering  defined as shifts in attention from task related processing to task unrelated thoughts  is a ubiquitous phenomenon that has a negative influence on performance and productivity in many contexts  including learning  we propose that next generation learning technologies should have some mechanism to detect and respond to mind wandering in real time  towards this end  we developed a technology that automatically detects mind wandering from eye gaze during learning from instructional texts  when mind wandering is detected  the technology intervenes by posing just in time questions and encouraging re reading as needed  after multiple rounds of iterative refinement  we summatively compared the technology to a yoked control in an experiment with 104 par

In [323]:
# 7) remove numbers by either a white space or the word "number"
# again, print the first 1000 characters of the first document
# to check that you're doing the right thing
for i,doc in enumerate(documents):
    for x in range(0, 10):
        doc = doc.replace(str(x),'')
    documents[i] = doc

In [306]:
# 8) Remove the stop words below from our documents
# print the first 1000 characters of the first document
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
              'ourselves', 'you', 'your', 'yours', 'yourself', 
              'yourselves', 'he', 'him', 'his', 'himself', 'she', 
              'her', 'hers', 'herself', 'it', 'its', 'itself', 
              'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', 
              'these', 'those', 'am', 'is', 'are', 'was', 'were', 
              'be', 'been', 'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
              'but', 'if', 'or', 'because', 'as', 'until', 'while', 
              'of', 'at', 'by', 'for', 'with', 'about', 'against', 
              'between', 'into', 'through', 'during', 'before', 
              'after', 'above', 'below', 'to', 'from', 'up', 'down', 
              'in', 'out', 'on', 'off', 'over', 'under', 'again', 
              'further', 'then', 'once', 'here', 'there', 'when', 
              'where', 'why', 'how', 'all', 'any', 'both', 'each', 
              'few', 'more', 'most', 'other', 'some', 'such', 'no', 
              'nor', 'not', 'only', 'own', 'same', 'so', 'than', 
              'too', 'very', 's', 't', 'can', 'will', 
              'just', 'don', 'should', 'now']

for i,doc in enumerate(documents):
    doc = [w for w in doc.split() if w not in stop_words]
    doc = " ".join(doc)
    documents[i] = doc 
print(documents[12][:1000])

zone mitigating mind wandering computerized reading sidney k d mello caitlin mills robert bixler nigel bosch university notre dame haggar hall notre dame usa sdmello nd edu abstract mind wandering defined shifts attention task related processing task unrelated thoughts ubiquitous phenomenon negative influence performance productivity many contexts including learning propose next generation learning technologies mechanism detect respond mind wandering real time towards end developed technology automatically detects mind wandering eye gaze learning instructional texts mind wandering detected technology intervenes posing time questions encouraging re reading needed multiple rounds iterative refinement summatively compared technology yoked control experiment participants key dependent variable performance post reading comprehension assessment results suggest technology successful correcting comprehension deficits attributed mind wandering d sigma specific conditions thereby highlighting po

In [324]:
# 9) remove words with one and two characters (e.g., 'd', 'er', etc.)
# print the first 1000 characters of the first document
for i,doc in enumerate(documents):
    #documents is a list of strings
    new_d = []
    for word in doc.split():
        if len(word) > 2: 
            new_d.append(word)
    doc = " ".join(new_d)
    documents[i] = doc
print(documents[12][:1000])

zone out more mitigating mind wandering during computerized reading sidney mello caitlin mills robert bixler nigel bosch university notre dame haggar hall notre dame usa sdmello edu abstract mind wandering defined shifts attention from task related processing task unrelated thoughts ubiquitous phenomenon that has negative influence performance and productivity many contexts including learning propose that next generation learning technologies should have some mechanism detect and respond mind wandering real time towards this end developed technology that automatically detects mind wandering from eye gaze during learning from instructional texts when mind wandering detected the technology intervenes posing just time questions and encouraging reading needed after multiple rounds iterative refinement summatively compared the technology yoked control experiment with participants the key dependent variable was performance post reading comprehension assessment our results suggest that the te


### Putting it all together

In [325]:
# 10) package all of your work above into a function that cleans a given document

def clean_list_of_documents(documents):
    
    cleaned_docs = []
    for i,doc in enumerate(documents):
        new_d = []
        for word in doc.split():
            if len(word) > 2: 
                new_d.append(word)
        doc = " ".join(new_d)
        documents[i] = doc
    for i,doc in enumerate(documents):
        for x in range(0, 10):
            doc = doc.replace(str(x),'')
            documents[i] = doc
    for i,doc in enumerate(documents):
        doc = [w for w in doc.split() if w not in stop_words]
        doc = " ".join(doc)
        documents[i] = doc 
    for doc in documents: 
        start = doc.rfind('\nabstract\n')+len('abstract')
        end = doc.rfind('reference')
        doc = abstract.append(doc[start:end])
    return cleaned_docs

In [326]:
# 11a) reimport your raw data using the code in 2)
documents = []
for paper in papers:
    file=open(paper,'r')
    documents.append(file.read())
documents[12]
        
# 11b) clean your files using the function above
clean_list_of_documents(documents)

# 11c) print the first 1000 characters of the first document
print(documents[11][:1000])

assessing computer literacy adults low literacy skills andrew olney institute intelligent systems university memphis memphis, aolney@memphis.edu daphne greenberg department educational psychology, special education, communication disorders georgia state university atlanta, dgreenberg@gsu.edu abstract adaptive learning technologies hold great promise improving reading skills adults low literacy, adults low literacy skills typically low computer literacy skills. order determine whether adults low literacy skills would able use intelligent tutoring system reading comprehension, adapted task computer literacy assessment delivered adults reading skills rd th grade levels. paper presents four analyses data. first, report pass/fail data natively exported assessment particular computer-based tasks. second, undertook goms analysis computer-based task, predict task completion time skilled user, found negatively correlated proportion correct item, r() −., .. third, used goms task decomposition de

## Step 3 - Build your list of vocabulary

This list of words (i.e., the vocabulary) is going to become the columns of your matrix.

In [327]:
import math
import numpy as np

12) Describe why we need to figure out the vocabulary used in our corpus (refer back to Sherin's paper, and explain in your own words): 
We can use the vocabulary to compute vectors for a passage, meaning we can see which words are used most in conjunction with others and hold the most meaning for large bodies of work and documents. 

In [311]:
documents[1]

'grade prediction temporal course-wise influence zhiyun ren computer science george mason university university drive, fairfax, zren@gmu.edu xia ning computer information science indiana university purdue university indianapolis university blvd, indianapolis, huzefa rangwala computer science george mason university university drive, fairfax, rangwala@cs.gmu.edu xning@cs.iupui.edu abstract critical need develop new educational technology applications analyze data collected universities ensure students graduate timely fashion years); well prepared jobs respective fields study. paper, present novel approach analyzing historical educational records large, public university perform next-term grade prediction; i.e., estimate grades student get course he/she enroll next term. accurate next-term grade prediction holds promise better student degree planning, personalized advising automated interventions ensure students stay track chosen degree program graduate time. present factorization-based 

In [328]:
# 13) create a function that takes in a list of documents
# and returns a set of unique words. Make sure that you
# sort the list alphabetically before returning it. 
def get_vocabulary(documents):
    voc = []
    for i,doc in enumerate(documents):
        for word in doc.split(): 
            voc.append(word)
    voc = list(set(voc))
    return voc

# Then print the length of your vocabulary (it should be 
# around 5500 words)
#print(get_vocabulary(documents))
print(len(get_vocabulary(documents)))

12485


In [ ]:
# 14) what was the size of Sherin's vocabulary? 
too large but 12485

## Step 4 - transform your documents into 100-words chunks

In [340]:
# 15) create a function that takes in a list of documents
# and returns a list of 100-words chunk 
# (with a 25 words overlap between them)
# Optional: add two arguments, one for the number of words
# in each chunk, and one for the overlap size
# Advice: combining all the documents into one giant string
# and splitting it into separate words will make your life easier!
chunks = []
with open('Papers/papers*.txt', 'w') as outfile:
    for doc in documents:
        with open('Papers/papers*.txt') as infile:
            for line in infile:
                outfile.write(line)
def chunks(documents):
    for doc in documents: 
        n = 100
        chunks = [[i * n:(i + 1) * n] for i in range((len(documents) + n - 1) // n )]
        print(chunk) 

In [329]:
# 16) create a for loop to double check that each chunk has 
# a length of 100
# Optional: use assert to do this check
for chunk in chunks:
     assert (len(chunk) == 100),"length is not equal to 100, length is " + str(len(chunk))

NameError: name 'chunks' is not defined

In [339]:
# 17) print the first chunk, and compare it to the original text.
# does that match what Sherin describes in his paper?
print(chunk[0])

TypeError: 'function' object is not subscriptable

In [ ]:
# 18) how many chunks did Sherin have? What does a chunk become 
# in the next step of our topic modeling algorithm? 


In [23]:
# 19) what are some other preprocessing steps we could do 
# to improve the quality of the text data? Mention at least 2.


In [25]:
# 20) in your own words, describe the next steps of the 
# data modeling algorithms (listed below):


## Step 5 - Vector and Matrix operations

## Step 6 - Weight word frequency

## Step 7 - Matrix normalization

## Step 8 - Deviation Vectors

## Step 9 - Clustering

## Step 10 - Visualizing the results

## Final Step - Putting it all together: 

In [17]:
# in python code, our goal is to recreate the steps above as functions
# so that we can just one line to run topic modeling on a list of 
# documents: 
def ExtractTopicsVSM(documents, numTopics):
    ''' this functions takes in a list of documents (strings), 
        runs topic modeling (as implemented by Sherin, 2013)
        and returns the clustering results, the matrix used 
        for clustering a visualization '''
    
    # step 2: clean up the documents
    documents = clean_list_of_documents(documents)
    
    # step 3: let's build the vocabulary of these docs
    vocabulary = get_vocabulary(documents)
    
    # step 4: we build our list of 100-words overlapping fragments
    documents = flatten_and_overlap(documents)
    
    # step 5: we convert the chunks into a matrix
    matrix = docs_by_words_matrix(documents, vocabulary)
    
    # step 6: we weight the frequency of words (count = 1 + log(count))
    matrix = one_plus_log_mat(matrix, documents, vocabulary)
    
    # step 7: we normalize the matrix
    matrix = normalize(matrix)
    
    # step 8: we compute deviation vectors
    matrix = transform_deviation_vectors(matrix, documents)
    
    # step 9: we apply a clustering algorithm to find topics
    results_clustering = cluster_matrix(matrix)
    
    # step 10: we create a visualization of the topics
    visualization = visualize_clusters(results_clustering, vocabulary)
    
    # finally, we return the clustering results, the matrix, and a visualization
    return results_clustering, matrix, visualization